In [1]:
import csv
from cobra.io import read_sbml_model, write_sbml_model
from copy import copy
from dotenv import find_dotenv
import os
from os.path import dirname
from functions import *

In [2]:
MODEL_PATH = "models/yeast-GEM.xml"

In [3]:
model = read_yeast_model(MODEL_PATH) # loading

In [4]:
write_yeast_model(model, "model_yeast8.xml")   # saving

# The Yeast8 model at a glance
`Lu, H., Li, F., Sánchez, B.J. et al. A consensus S. cerevisiae metabolic model Yeast8 and its ecosystem for comprehensively probing cellular metabolism. Nat Commun 10, 3586 (2019).`https://doi.org/10.1038/s41467-019-11581-3

### Media composition

In [7]:
media_comp = list(model.medium)
for i in range (0, len(media_comp)):
    print(model.medium[media_comp[i]], model.reactions.get_by_id(media_comp[i]).name)
    if 'glucose' in model.reactions.get_by_id(media_comp[i]).name:
        carbon_exchange_id = media_comp[i]
        glucose_id = model.reactions.get_by_id(carbon_exchange_id).reaction[:8] # carbon source ID

1000.0 ammonium exchange
1.0 D-glucose exchange
1000.0 H+ exchange
1000.0 iron(2+) exchange
1000.0 oxygen exchange
1000.0 phosphate exchange
1000.0 potassium exchange
1000.0 sodium exchange
1000.0 sulphate exchange
1000.0 water exchange
1000.0 chloride exchange
1000.0 Cu2(+) exchange
1000.0 Mn(2+) exchange
1000.0 Zn(2+) exchange
1000.0 Mg(2+) exchange
1000.0 Ca(2+) exchange


### Genes, metabolites and reactions

In [8]:
print("The Yeast8 Saccharomyces cerevisiae model contains "+str(len(model.genes))+" genes, "+str(len(model.metabolites))+" metabolites and "+str(len(model.reactions))+" reactions.")

The Yeast8 Saccharomyces cerevisiae model contains 1150 genes, 2742 metabolites and 4058 reactions.


In [9]:
print(model.compartments)

{'ce': 'cell envelope', 'c': 'cytoplasm', 'e': 'extracellular', 'm': 'mitochondrion', 'n': 'nucleus', 'x': 'peroxisome', 'r': 'endoplasmic reticulum', 'g': 'Golgi', 'lp': 'lipid particle', 'v': 'vacuole', 'rm': 'endoplasmic reticulum membrane', 'vm': 'vacuolar membrane', 'gm': 'Golgi membrane', 'mm': 'mitochondrial membrane'}


In [10]:
for metabolite in model.metabolites.query('glucose', 'name'):
    print(metabolite.name,metabolite.id)

D-glucose glc__D_c
D-glucose glc__D_e
D-glucose glc__D_v
D-glucose 1-phosphate g1p_c
D-glucose 6-phosphate g6p_c
UDP-D-glucose udpg_c
D-glucose glc__D_r
2-deoxy-D-glucose 6-phosphate 2doxg6p_c
2-deoxy-D-glucose 2dglc_c
UDP-D-glucose udpg_r
D-glucose 1-phosphate g1p_e
D-glucose 6-phosphate g6p_e


In [11]:
print("The growth rate of Yeast8 is: "+str(model.optimize().objective_value)[:5])
biomass_id = str(model.objective).split(' ')[0].split('*')[1]

The growth rate of Yeast8 is: 0.083


## Reactions in the native pathway leading to our target product

In [14]:
for reactions in model.reactions.query('ferrochelatase', 'name'):
    print(reactions.name,reactions.id)
    ferrochelatase_id = str(reactions.id)
model.reactions.get_by_id(ferrochelatase_id)

ferrochelatase FCLTm


Reaction identifier,FCLTm
Name,ferrochelatase
Memory address,0x01f08bdd6c48
Stoichiometry,fe2_m + ppp9_m --> 2.0 h_m + pheme_m iron(2+) + protoporphyrin --> 2.0 H+ + ferroheme b
GPR,YOR176W
Lower bound,0.0
Upper bound,1000.0


In [15]:
medium = model.medium
with model:
    solution = model.optimize()
    print("Yeast8 maximum theoretical biomass productivity:", solution.fluxes[biomass_id], '/h')
    model.medium = medium
    model.objective = model.reactions.get_by_id(ferrochelatase_id)
    heme_production = model.optimize().objective_value
    print("Yeast8t maximum theoretical productivity of heme ", heme_production, '[mmol gDW^-1 h^-1]')

Yeast8 maximum theoretical biomass productivity: 0.08374770604149177 /h
Yeast8t maximum theoretical productivity of heme  8.374770583004913e-08 [mmol gDW^-1 h^-1]


## Enumerated pathway to heme

In [16]:
heme_1 = model.reactions.get_by_id('ALASm')
heme_2 = model.reactions.get_by_id('PPBNGS')
heme_3 = model.reactions.get_by_id('HMBS')
heme_4 = model.reactions.get_by_id('UPP3S') 
heme_12 = model.reactions.get_by_id('UPPDC1') 
heme_13 = model.reactions.get_by_id('CPPPGO')
heme_14 = model.reactions.get_by_id('PPPGOm') # second to last reaction in the heme biosynthesis
heme_15 = model.reactions.get_by_id('FCLTm')
heme_pathway = [heme_1,heme_2,heme_3,heme_4,heme_12,heme_13,heme_14,heme_15]

In [17]:
for i in range (0,len(heme_pathway)):
    with model:
        model.objective = heme_pathway[-1] # calculates the flux with heme as the objective
        print(model.optimize().fluxes[heme_pathway[i].id], heme_pathway[i].id, heme_pathway[i].reaction, heme_pathway[i].name)

6.699816466403931e-07 ALASm gly_m + h_m + succoa_m --> 5aop_m + co2_m + coa_m 5-aminolevulinate synthase
3.3499082332019654e-07 PPBNGS 2.0 5aop_c --> 2.0 h2o_c + h_c + ppbng_c porphobilinogen synthase
8.374770583004913e-08 HMBS h2o_c + 4.0 ppbng_c --> hmbil_c + 4.0 nh4_c hydroxymethylbilane synthase
8.374770583004913e-08 UPP3S hmbil_c --> h2o_c + uppg3_c uroporphyrinogen-III synthase
8.374770583004913e-08 UPPDC1 4.0 h_c + uppg3_c --> 4.0 co2_c + cpppg3_c uroporphyrinogen decarboxylase (uroporphyrinogen III)
8.374770583004913e-08 CPPPGO cpppg3_c + 2.0 h_c + o2_c --> 2.0 co2_c + 2.0 h2o_c + pppg9_c coproporphyrinogen oxidase (O2 required)
4.187385291502457e-08 PPPGOm 3.0 o2_m + 2.0 pppg9_m --> 6.0 h2o_m + 2.0 ppp9_m protoporphyrinogen oxidase
8.374770583004913e-08 FCLTm fe2_m + ppp9_m --> 2.0 h_m + pheme_m ferrochelatase


# Pseudopathway for recombinant haemoglobin

In [18]:
from cobra import Reaction, Metabolite

In [19]:
add_rhb(model, "model_yeast8_rhb.xml")

In [20]:
MODEL_PATH = "model_yeast8_rhb.xml"
model = read_yeast_model(MODEL_PATH) # loading

## Enumerated pathway to recombinant haemoglobin in yeast, starting with glycine

In [21]:
rhb_pathway = get_rhb_pathway(model)

In [22]:
print(rhb_pathway) # now we can work with the model in other notebooks (i.e. media, mutants, etc.)

[<Reaction ALASm at 0x1f08ef27a88>, <Reaction PPBNGS at 0x1f08f0b8308>, <Reaction HMBS at 0x1f08eff7d48>, <Reaction UPP3S at 0x1f08f14afc8>, <Reaction UPPDC1 at 0x1f08f13ec08>, <Reaction CPPPGO at 0x1f08ef9fbc8>, <Reaction PPPGOm at 0x1f08f0c1bc8>, <Reaction FCLTm at 0x1f08efbe388>, <Reaction rHb at 0x1f09039b908>]


In [23]:
for i in range (0,len(rhb_pathway)):
    with model:
        model.objective = rhb_pathway[-1] # calculates the flux with heme as the objective
        print(model.optimize().fluxes[rhb_pathway[i].id], rhb_pathway[i].id, rhb_pathway[i].reaction, rhb_pathway[i].name)

0.5226848222438156 ALASm gly_m + h_m + succoa_m --> 5aop_m + co2_m + coa_m 5-aminolevulinate synthase
0.2613424111219078 PPBNGS 2.0 5aop_c --> 2.0 h2o_c + h_c + ppbng_c porphobilinogen synthase
0.06533560278047695 HMBS h2o_c + 4.0 ppbng_c --> hmbil_c + 4.0 nh4_c hydroxymethylbilane synthase
0.06533560278047695 UPP3S hmbil_c --> h2o_c + uppg3_c uroporphyrinogen-III synthase
0.06533560278047695 UPPDC1 4.0 h_c + uppg3_c --> 4.0 co2_c + cpppg3_c uroporphyrinogen decarboxylase (uroporphyrinogen III)
0.06533560278047695 CPPPGO cpppg3_c + 2.0 h_c + o2_c --> 2.0 co2_c + 2.0 h2o_c + pppg9_c coproporphyrinogen oxidase (O2 required)
0.03266780139023848 PPPGOm 3.0 o2_m + 2.0 pppg9_m --> 6.0 h2o_m + 2.0 ppp9_m protoporphyrinogen oxidase
0.06533560278047695 FCLTm fe2_m + ppp9_m --> 2.0 h_m + pheme_m ferrochelatase
0.01633390069511924 rHb 72.0 ala__L_c + 12.0 arg__L_c + 20.0 asn__L_c + 30.0 asp__L_c + 6.0 cys__L_c + 8.0 gln__L_c + 24.0 glu__L_c + 40.0 gly_c + 38.0 his__L_c + 72.0 leu__L_c + 44.0 lys_

In [24]:
medium = model.medium
with model:
    model.medium = medium
    model.objective = model.reactions.rHb
    rhb_production = model.optimize().objective_value
    print("Max. recombinant haemoglobin production: "+str(rhb_production)[:6]+" [mmol gDW^-1 h^-1]")
    print("Theoretical max. yield: "+ str(rhb_production / (-1*model.reactions.EX_glc__D_e.flux))[:7]+" [mmol-rHb / mmol-glucose]")

Max. recombinant haemoglobin production: 0.0163 [mmol gDW^-1 h^-1]
Theoretical max. yield: 0.00163 [mmol-rHb / mmol-glucose]


# Rationale-based genetic modifications

### Heme activated protein, HAP1 (YLR256W)

In the lab, *Saccharomyces cerevisiae* activate the biosynthesis of heme when sensing oxygen though unknown mechanisms. 

The heme then goes on to activate mitocondrial respiration though the **Hap1** protein (https://www.yeastgenome.org/locus/S000004246). The Hap1p also supresses anaerobic metabolism though activation of the transcriptional repressor **Rox1p** (https://www.yeastgenome.org/locus/S000006269).

`Martinez, J. L., Petranovic, D., and Nielsen, J. “Heme metabolism in stress regulation andprotein production: From Cinderella to a key player”. In: Bioengineered 7.2 (Apr. 2016) http://dx.doi.org/10.1080/21655979.2015.1126016`

Ideally, we would knock out either HAP1 or ROX1 with the aim of increasing endogenous biosynthesis of heme:

In [30]:
for gene in model.genes.query('HAP', 'name'):
    print(gene.name,gene.id)

In [31]:
for gene in model.genes.query('ROX', 'name'):
    print(gene.name,gene.id)

But since both of these proteins are transcription factors, and do not catalyse enzymatic reactions in the GSMM-sense, they are not to be found in the model.

Instead we may aim for some of the proteins that are upregulated by **Hap1p** and **Rox1p**. 


**These proteins include**
* The cytochromes for mitocondrial respiration: CYC1, CYB2, CYC7, QCR2, CYT1
* Other enzymes involved in the oxidative stress response: YHB1, CTT1, SOD2

Let's scan the model for these genes.

In [41]:
ls = ['CYC1', 'CYB2', 'CYC7', 'QCR2', 'CYT1']
ls1 = []
print("The cytochromes for mitocondrial respiration")
for i in range(0, len(ls)):
    for gene in model.genes.query(ls[i], 'name'):
        print(gene.name,gene.id)
        ls1.append(gene.id)

print("Other enzymes involved in the oxidative stress response")
ls = ['YHB1', 'CTT1', 'SOD2']
ls2 = []
for i in range(0, len(ls)):
    for gene in model.genes.query(ls[i], 'name'):
        print(gene.name,gene.id)
        ls2.append(gene.id)

The cytochromes for mitocondrial respiration
CYC1 YJR048W
CYB2 YML054C
CYC7 YEL039C
QCR2 YPR191W
CYT1 YOR065W
Other enzymes involved in the oxidative stress response
YHB1 YGR234W
CTT1 YGR088W
SOD2 YHR008C


In [42]:
test_knockout(model, ls1)

---Before knockout---
Growth rate: 0.8637367930478737
Rate of hemoglobin production with hemoglobin as the objective:  0.016333900695119228  [mmol / gDW / h]
Maximum theoretical yield of hemoglobin on glucose  0.0016333900695119229 [mmol-rHb / mmol-glucose]
---After knockout---
Growth rate: 0.005583952165107507
Rate of hemoglobin production: 0.005583952165107507  [mmol / gDW / h]
Yield for hemoglobin on sugar 0.0005583952165107507 [mmol-rHb / mmol-glucose]
--------------------


Knocking out all the cytochromes for mitocondrial respiration significantly decreased the growth rate. It's like the model ignores fermentation as an option.

We also see a significant drop in the rate of hemoglobin production, which was surprising as experimental data would suggest the opposite to be true `Martinez, J. L. et al. (2016) http://dx.doi.org/10.1080/21655979.2015.1126016`.

In [43]:
test_knockout(model, ls2)

---Before knockout---
Growth rate: 0.8637367930478737
Rate of hemoglobin production with hemoglobin as the objective:  0.016333900695119228  [mmol / gDW / h]
Maximum theoretical yield of hemoglobin on glucose  0.0016333900695119229 [mmol-rHb / mmol-glucose]
---After knockout---
Growth rate: 0.016333900695119266
Rate of hemoglobin production: 0.016333900695119266  [mmol / gDW / h]
Yield for hemoglobin on sugar 0.0016333900695119266 [mmol-rHb / mmol-glucose]
--------------------


Knocking out the proteins involed in the oxidative stress response also appear to significantly decreased the growth rate, but not as much as when we knocked out all the cytochromes for mitocondrial respiration.

Here we see no change in the rate of hemoglobin production.

### YFH1
YFH1 is a hemoglobin-like protein in *Saccharomyces cerevisiae*, that catalyses the conversion of iron(II) to iron(III) (https://www.genome.jp/kegg-bin/show_pathway?sce00860)

We need iron(II) for the heme-groups in hemoglobin, so ideally we don't want to waste any in this pathway.

As we say earlier in the media composition, iron(II) is by no means considered a limiting factor. 
However, in the real-world, the uptake and utilisation of iron(II) could very well be an important facotor in the synthesis of heme.
Therefore, we don't expect to see an increase in the yield of hemoglobin from this knockout, but rather we're interested in whether the knockout will impact the growth of our cell factory.

Let's try knocking out YFH1 (https://www.yeastgenome.org/locus/S000002278).

In [44]:
for gene in model.genes.query('YFH1', 'name'):
    print(gene.name,gene.id,gene.reactions)

YFH1 YDL120W frozenset({<Reaction FEROm at 0x1f0900c0f08>})


In [45]:
test_knockout(model, 'YDL120W')

---Before knockout---
Growth rate: 0.8637367930478737
Rate of hemoglobin production with hemoglobin as the objective:  0.016333900695119228  [mmol / gDW / h]
Maximum theoretical yield of hemoglobin on glucose  0.0016333900695119229 [mmol-rHb / mmol-glucose]
---After knockout---
YFH1  knocked out.
Growth rate: 0.016333900695119228
Rate of hemoglobin production: 0.016333900695119228  [mmol / gDW / h]
Yield for hemoglobin on sugar 0.0016333900695119229 [mmol-rHb / mmol-glucose]
--------------------


Just as we suspected, there was not change in the yield of hemoglobin. However, the growth-rate was significantly decreased. 

Rather than a knockout, it would be interesting to investingate just how significantly a down-regulation of the gene-expression would impact the growth rate. 

### Other draining pathways

The enzymes COX10 and COX15 does appear to drain heme to the Heme O and Heme A metabolites 

CYC3 and CYT2 also appear to drain heme into cytochrome C.

`https://www.genome.jp/kegg-bin/show_pathway?sce00860`

#### COX10

In [54]:
for gene in model.genes.query('COX10', 'name'):
    print(gene.name,gene.id,gene.reactions)
    test_knockout(model, gene.id)

COX10 YPL172C frozenset({<Reaction HEMEOSm at 0x1f08efe9408>})
---Before knockout---
Growth rate: 0.8637367930478737
Rate of hemoglobin production with hemoglobin as the objective:  0.016333900695119228  [mmol / gDW / h]
Maximum theoretical yield of hemoglobin on glucose  0.0016333900695119229 [mmol-rHb / mmol-glucose]
---After knockout---
COX10  knocked out.
Growth rate: 0.01633390069511925
Rate of hemoglobin production: 0.01633390069511925  [mmol / gDW / h]
Yield for hemoglobin on sugar 0.0016333900695119248 [mmol-rHb / mmol-glucose]
--------------------


#### COX15

In [53]:
for gene in model.genes.query('COX15', 'name'):
    print(gene.name,gene.id,gene.reactions)
    test_knockout(model, gene.id)

COX15 YER141W frozenset({<Reaction HEMEOMOm at 0x1f08efe2cc8>})
---Before knockout---
Growth rate: 0.8637367930478737
Rate of hemoglobin production with hemoglobin as the objective:  0.016333900695119228  [mmol / gDW / h]
Maximum theoretical yield of hemoglobin on glucose  0.0016333900695119229 [mmol-rHb / mmol-glucose]
---After knockout---
COX15  knocked out.
Growth rate: 0.016333900695119228
Rate of hemoglobin production: 0.016333900695119228  [mmol / gDW / h]
Yield for hemoglobin on sugar 0.0016333900695119229 [mmol-rHb / mmol-glucose]
--------------------


#### CYC3

In [56]:
for gene in model.genes.query('CYC3', 'name'):
    print(gene.name,gene.id,gene.reactions)
    test_knockout(model, gene.id)

CYC3 YAL039C frozenset({<Reaction HEMELm at 0x1f090037c08>})
---Before knockout---
Growth rate: 0.8637367930478737
Rate of hemoglobin production with hemoglobin as the objective:  0.016333900695119228  [mmol / gDW / h]
Maximum theoretical yield of hemoglobin on glucose  0.0016333900695119229 [mmol-rHb / mmol-glucose]
---After knockout---
CYC3  knocked out.
Growth rate: 0.016333900695119228
Rate of hemoglobin production: 0.016333900695119228  [mmol / gDW / h]
Yield for hemoglobin on sugar 0.0016333900695119229 [mmol-rHb / mmol-glucose]
--------------------


#### CYT2

In [58]:
for gene in model.genes.query('CYT2', 'name'):
    print(gene.name,gene.id,gene.reactions)
    test_knockout(model, gene.id)

CYT2 YKL087C frozenset({<Reaction HEMELm at 0x1f090037c08>})
---Before knockout---
Growth rate: 0.8637367930478737
Rate of hemoglobin production with hemoglobin as the objective:  0.016333900695119228  [mmol / gDW / h]
Maximum theoretical yield of hemoglobin on glucose  0.0016333900695119229 [mmol-rHb / mmol-glucose]
---After knockout---
CYT2  knocked out.
Growth rate: 0.016333900695119228
Rate of hemoglobin production: 0.016333900695119228  [mmol / gDW / h]
Yield for hemoglobin on sugar 0.0016333900695119229 [mmol-rHb / mmol-glucose]
--------------------
